In [ ]:
!pip3 install pytorch_forecasting wandb pandas

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'pytorch_stats_own_data.ipynb'
os.environ['WANDB_API_KEY'] = 'f59f037973c4de4681bc9330ea3a477cb0328ca5'

In [3]:
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch

from pytorch_forecasting import Baseline, NBeats, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE

from pytorch_lightning.loggers import WandbLogger
# wandb_logger = WandbLogger(project="Digital-Energy")

import os
AVAILABLE_GPUS = torch.cuda.device_count()
AVAILABLE_CPUS = os.cpu_count()

print(f"Available GPUs: {AVAILABLE_GPUS}")
print(f"Available CPUs: {AVAILABLE_CPUS}")

Available GPUs: 2
Available CPUs: 32


# Own dataset

We load in all the data from the london dataset for fast testing

In [ ]:
# Testing on 1 file

file = "./Data/london/household_MAC000002.csv"
data = pd.read_csv(file, parse_dates=["DateTime"])
data = data.drop(columns=["stdorToU", "timeOfDay", "Unnamed: 0"])
data = data.rename(columns={"KWH/hh (per half hour) ": "KWH"})
data["time_idx"] = data.index
# drop missing values of KWH
data = data.dropna(subset=["KWH"]).reset_index(drop=True)
data.head()

## Read all files in data folder

here we read all the CSV files in the data folder and create our dataframe

This dataframe has more then a Billion rows (with a B) so it will takes some time. 13 min on a 2018 macbook pro 15'

In [ ]:
import glob
# load all csv files in london folder into 1 dataframe
data = pd.concat([pd.read_csv(f) for f in sorted(glob.glob("data/london/*.csv"))])
data

Clean dataframe. Here we drop the columns that are taking up space and rename the WKH column for easy querying

In [ ]:
data = data.drop(columns=["stdorToU", "timeOfDay", "Unnamed: 0"])
data = data.rename(columns={"KWH/hh (per half hour) ": "KWH"})
data["time_idx"] = data.index
# drop missing values of KWH
data = data.dropna(subset=["KWH"]).reset_index(drop=True)

## Save dataframe in more performant format (parquet)

Due to the size of the data i did some research on the best format to save the data.

here the results show that the parquet and feather formats are the fastest to read and write. 


with parquet being the fastest overall while taking up the least disk space.

In [ ]:
## Comented for safety
# data.to_parquet("data/ldn_df.parquet")
# data.to_parquet("data/ldn_df_small.parquet")

In [ ]:
# data = pd.read_parquet("data/ldn_df.parquet")
data = pd.read_parquet("../../Data/ldn_df_small.parquet")
data

In [ ]:
data["DateTime"] = pd.to_datetime(data["DateTime"])

# Adding time_idx Column

we add this column to the dataframe so pytorch-forecasting can use the time_idx

In [ ]:
# find all unique households
households = data["LCLid"].unique()

# find highest count of "LCLid" unique values and create a range from it
rows_of_households = data["LCLid"].value_counts().sort_index()
rows_of_households = [range(1, x + 1) for x in rows_of_households.values]

In [ ]:
# add households and ranges to a dictionary
dict = {k:v for k,v in zip(households, rows_of_households)}

In [ ]:
def fill_output(r):
    r['time_idx_2'] = dict[r.iloc[0]['LCLid']]
    return r


df = data.groupby('LCLid').apply(fill_output)

In [ ]:
x = []
for i in rows_of_households:
    x += [i[-1]]

min(x)

Before starting training, we need to split the dataset into a training and validation TimeSeriesDataSet.

here are the parameters used for the TimeSeriesDataSet

In [ ]:
# create dataset and dataloaders
max_prediction_length = 48
max_encoder_length = max_prediction_length * 5

training_cutoff = 19523 - max_prediction_length

context_length = max_encoder_length
prediction_length = max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="KWH",
    categorical_encoders={"LCLid": NaNLabelEncoder().fit(data.LCLid)},
    group_ids=["LCLid"],
    # only unknown variable is "KWH" - and N-Beats can also not take any additional variables
    time_varying_unknown_reals=["KWH"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
    allow_missing_timesteps=True,
)

validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training_cutoff + 1)
batch_size = 400
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=AVAILABLE_CPUS)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=AVAILABLE_CPUS)

## Calculate baseline error

Our baseline model predicts future values by repeating the last know value. The resulting SMAPE is disappointing and should not be easy to beat.

In [ ]:
# calculate baseline absolute error
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
SMAPE()(baseline_predictions, actuals)

# Train network

Finding the optimal learning rate using [PyTorch Lightning](https://pytorch-lightning.readthedocs.io/) is easy. The key hyperparameter of the NBeats model are the widths. Each denotes the width of each forecasting block. By default, the first forecasts the trend, while the second forecasts seasonality.

![test](https://miro.medium.com/max/836/1*1If8JU4JwFAta1kjMjkTQg.png)

In [ ]:
pl.seed_everything(42)
trainer = pl.Trainer(gpus=1,gradient_clip_val=0.01, logger=wandb_logger)
net = NBeats.from_dataset(training, learning_rate=3e-2, weight_decay=1e-2, widths=[32, 512], backcast_loss_ratio=0.1)

In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(net, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()
net.hparams.learning_rate = res.suggestion()

## Fit the model

In [ ]:
import time
t0 = time.time()


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.01,
    callbacks=[early_stop_callback],
    limit_train_batches=30,
    logger=wandb_logger
)


net = NBeats.from_dataset(
    training,
    learning_rate=net.hparams.learning_rate if net.hparams.learning_rate else 0.02,
    log_interval=10,
    log_val_interval=1,
    weight_decay=1e-2,
    widths=[32, 512],
    backcast_loss_ratio=1.0
)

trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

print(f"{time.time() - t0} seconds")

# Evaluate Results

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = NBeats.load_from_checkpoint(best_model_path)

We predict on the validation dataset with predict() and calculate the error which is well below the baseline error

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)
(actuals - predictions).abs().mean()

Looking at random samples from the validation set is always a good way to understand if the forecast is reasonable - and it is!

In [ ]:
raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
for idx in range(3):  # plot 10 examples
    best_model.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True)

## Interpret model

We can ask PyTorch Forecasting to decompose the prediction into seasonality and trend with plot_interpretation(). This is a special feature of the NBeats model and only possible because of its unique architecture. The results show that there seem to be many ways to explain the data and the algorithm does not always chooses the one making intuitive sense. This is partially down to the small number of time series we trained on (100). But it is also due because our forecasting period does not cover multiple seasonalities.

In [ ]:
for idx in range(1):  # plot 10 examples
    best_model.plot_interpretation(x, raw_predictions, idx=idx)